In [1]:
import praw
import re
import string
import time
import os
from os import path
import pandas as pd
from datetime import datetime, timezone

In [2]:
# Criar arquivo CSV
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'dataReddit.csv')

    if not path.exists(path.dirname(basePath)):
        print("O diretório 'db' não existe.")
        return None

    if not path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('index,data,ID,Titulo,Autor,Flair,Upvotes,Dowvotes,URL,Post\n')

    return basePath


In [ ]:
# Conectar ao Reddit usando o PRAW
reddit = praw.Reddit(
    client_id="XF25-zDXTONeO2NeHIuwRQ",
    client_secret="KF3wx4XWQTI2-jCsw5KN2X1GaM4tCw",
    user_agent="web_scraper",
)

subreddit = reddit.subreddit("brasilivre")



Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [4]:
def cleanSentence(phrase):
    # Convertendo para minúsculas
    phrase = phrase.lower()

    # Removendo URLs
    phrase = re.sub(r'http\S+|www\S+|https\S+', '', phrase, flags=re.MULTILINE)

    # Removendo números
    phrase = re.sub(r'\d+', '', phrase)

    # Removendo pontuação
    phrase = phrase.translate(str.maketrans('', '', string.punctuation))

    # Removendo espaços extras
    phrase = phrase.strip()

    # Removendo múltiplos espaços
    phrase = re.sub(r'\s+', ' ', phrase)

    return phrase
    

In [5]:

base = []
maxExtract = 3000
loteSize = 250

for i in range(0, maxExtract, loteSize):
    print(f'coletando de {i} à {i+loteSize}')

    for submission in subreddit.new(limit=loteSize):
        print(f"Título: {submission.title}")
        
        post_date = datetime.fromtimestamp(submission.created_utc, tz=timezone.utc).strftime('%d/%m/%y')

        # Coleta de dados
        title = cleanSentence(submission.title) if submission.title else 'N/A'
        post_text = cleanSentence(submission.selftext) if submission.selftext else 'N/A'

        # Tratamento de dados
        autor = submission.author if submission.author else 'N/A'
        flair = submission.link_flair_text if submission.link_flair_text else 'N/A'
        print(f"Flair: {submission.link_flair_text}")

            
        base.append([
            post_date,
            submission.id,
            title,
            autor,
            flair,
            submission.score,
            submission.downs,
            submission.url,
            post_text,
        ])
    
    time.sleep(1.5)

coletando de 0 à 250
Título: Termina hoje o pagamento do “Primeiro lote” auxílio emergencial para aqueles que se inscreveram do dia 7 à 10
Flair: None
Título: Mãe de Isis Helena afirma ter jogado o corpo da bebê no rio por que se desesperou ao perceber que a criança tinha morrido engasgada com leite
Flair: None
Título: Luciano Huck ‘toca na ferida’ de Bolsonaro, mas sofre duro castigo
Flair: None
Título: Urgente: avião cai e explode no Brasil; não sobrou nada
Flair: None
Título: Carolina Dieckmann celebra 21 anos do filho com Marcos Frota e fãs reagem: ‘a cara do pai’
Flair: None
Título: Menino de 10 anos trabalha sozinho para sustentar a família, uma história de partir o coração
Flair: None
Título: Caso Ísis Helena: Jenifer confessa que tirou a vidinha de Ísis, descanse em paz anjinho
Flair: None
Título: Alguns famosos que têm filhos e você nem sabia
Flair: None
Título: Vaza vídeo de Diego Alemão sendo algemado e colocado no camburão da PM
Flair: None
Título: Luto na música: cantor mo

In [6]:
# Criar DataFrame
df = pd.DataFrame(base, columns=["Data","ID", "Titulo", "Autor", "Flair", "Upvotes", "Dowvotes", "URL", "Post"])

# Substituir valores nulos
df.fillna('N/A', inplace=True)

# Verificar tipos de dados
df['Upvotes'] = df['Upvotes'].astype(int)
df['Dowvotes'] = df['Dowvotes'].astype(int)

# Caminho do CSV
csv_file_path = baseArchive()
df.to_csv(csv_file_path, mode='a', header=False, index=True, encoding='utf-8')


In [7]:
df['Flair'].unique()

array(['N/A'], dtype=object)